# VGDB Examples
This notebook highlights some key features of VGDB that make it a useful tool for quickly solving your trickiest errors.

A quick cheat sheet of common commands:

```bash
valgrind --vgdb=full --vgdb-error=1 --read-var-info=yes --leak-check=full ./example1 < ex1_input.txt

gdb example1

target remote | vgdb

monitor who_points_at <address> <length>
monitor xb <address>
monitor check_memory defined <address> <length>
monitor v.info last_error
monitor v.info all_errors
monitor leak_check
monitor leak_check full reachable any
monitor block_list
monitor v.kill
```

## Example Program
Here is a program which has issues which valgrind can easily pick up. It shows some common mistakes in memory management.

It's a simple program meant to read data into a space and then stores it in a small linked list structure.

In [ ]:
//% stdin: "input line 1"
//% stdin: "input line 2"
//% stdin: "input line 3"
//% stdin: "4th input line"
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>

#define MAXSTRINGSIZE 15

struct node {
    char *string;
    struct node *next;
};

struct node *new_node(){
    struct node *new_node = (struct node *) malloc(sizeof(struct node));
    new_node->string = (char *) malloc(sizeof(char) * MAXSTRINGSIZE);
    assert(new_node->string);
    return new_node;
}

void store_string(struct node *node, char *buffer){
    node->string = buffer;
}

int main(int argc, char **argv){
    char *buffer = (char *) malloc(sizeof(char) * MAXSTRINGSIZE);
    assert(buffer);
    struct node *head = new_node();
    while(scanf("%[^\n]\n", buffer) == 1){
        printf("%s\n", buffer);
        store_string(head, buffer);
    }
    printf("Head string: %s\n", head->string);
    printf("Next string: %s\n", head->next->string);
    printf("Next next string: %s\n", head->next->next->string);
    printf("Last string: %s\n", head->next->next->next->string);
    
    free(head->string);
    free(head);
    return 0;
}


### Expected Output
The expected last four lines of the output are:
```
Head string: 4th input line
Next string: input line 3
Next next string: input line 2
Last string: input line 1
```